In [1]:
from datasets import load_dataset
from transformers import DefaultDataCollator
from datasets import concatenate_datasets, DatasetDict
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator
import numpy as np
from collections import defaultdict


data_collator = DefaultDataCollator()
langs = [ "de", "en", "es", "hi"]


mlqa = {}

for lang1 in langs:
    for lang2 in langs:
        mlqa[f"{lang1}.{lang2}"] = load_dataset("mlqa", f"mlqa.{lang1}.{lang2}",trust_remote_code=True)

    
def prepare_validation_features(examples):
    
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1
        
        
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples    

def prepare_train_features(examples):
    
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    
    
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        
        sequence_ids = tokenized_examples.sequence_ids(i)

        
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                
                
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


from tqdm.auto import tqdm
import collections
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    
    predictions = collections.OrderedDict()

    
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    
    for example_index, example in enumerate(tqdm(examples)):
        
        feature_indices = features_per_example[example_index]

        valid_answers = []
        
        context = example["context"]
        
        for feature_index in feature_indices:
            
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            
            
            offset_mapping = features[feature_index]["offset_mapping"]

            
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    
                    
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            
            
            best_answer = {"text": "", "score": 0.0}
        
        
        predictions[example["id"]] = best_answer["text"]

    return predictions

mlqa.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/512 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4517 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/507 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1723 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/187 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4918 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/507 [00:00<?, ? examples/s]

In [2]:
model_name = "alon-albalak/xlm-roberta-base-xquad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


max_length = 384
doc_stride = 128


validation_datasets = []
for lang_pair, dataset in mlqa.items():
    if 'validation' in dataset:
        validation_datasets.append(dataset['validation'])
print("ge")

test_datasets = []
for lang_pair, dataset in mlqa.items():
    if 'test' in dataset:
        test_datasets.append(dataset['test'])

print("fd")

merged_train_dataset = concatenate_datasets(validation_datasets)


print("fdsfsdfsd")

merged_test_dataset = concatenate_datasets(test_datasets)

merged_test_dataset = merged_test_dataset.select(range(100))
print("fds")

train_processed = merged_train_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=merged_train_dataset.column_names,
    desc="Processing training dataset",
)


test_processed = merged_test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=merged_test_dataset.column_names,
    desc="Processing test dataset",
)

print(f"Training dataset size: {len(train_processed)}")
print(f"Test dataset size: {len(test_processed)}")

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

...

ge
fd
fdsfsdfsd
fds


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing training dataset:   0%|          | 0/6797 [00:00<?, ? examples/s]

Processing test dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Training dataset size: 8175
Test dataset size: 116


In [3]:

training_args = TrainingArguments(
    output_dir="./results",          
    evaluation_strategy="steps",    
    save_steps=500,                 
    eval_steps=500,                 
    logging_steps=100,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    learning_rate=3e-5,             
    num_train_epochs=3,             
    weight_decay=0.01,              
    save_total_limit=2,             
    fp16=True,                      
    dataloader_num_workers=4,       
    report_to="none"                
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_processed,
    eval_dataset=test_processed,
    data_collator=data_collator,  
    tokenizer=tokenizer           
)


trainer.train()


model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("Model fine-tuned and saved.")


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Model fine-tuned and saved.


In [4]:
!pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [5]:
from collections import defaultdict

mlqa_prep = defaultdict(dict)

def map_datasets(langs, split, prepare_features):
    for lang in langs:
        print(lang)
        mlqa_prep[lang][split] = mlqa[lang][split].map(prepare_features, batched=True, 
                                    remove_columns=mlqa[lang][split].column_names)

from evaluate import load

squad_metric = load("squad")

def compute_results(langs, split):
    results = {}
    for lang in langs:
        
        print(lang,langs)
        raw_predictions = trainer.predict(mlqa_prep[lang][split])

        
        mlqa_prep[lang][split].set_format(type=mlqa_prep[lang][split].format["type"], 
                        columns=list(mlqa_prep[lang][split].features.keys()))
        
        
        final_predictions = postprocess_qa_predictions(mlqa[lang][split], mlqa_prep[lang][split], raw_predictions.predictions)

        
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
        references = [{"id": ex["id"], "answers": ex["answers"]} for ex in mlqa[lang][split]]
        results[lang] = squad_metric.compute(predictions=formatted_predictions, references=references)
    return results


import pandas as pd

def results_df(results_dict, model):
    F1colname = "F1_" + model
    EMcolname = "EM_" + model
    dict_results = defaultdict(list)
    for lang, scores in results_dict.items():
        dict_results["lang"].append(lang)
        dict_results[F1colname].append(scores['f1'])
        dict_results[EMcolname].append(scores['exact_match'])

    avg_f1 = np.average(dict_results[F1colname])
    avg_em = np.average(dict_results[EMcolname])
    dict_results["lang"].append('avg')
    dict_results[F1colname].append(avg_f1)
    dict_results[EMcolname].append(avg_em)
    df_results = pd.DataFrame(dict_results).round(2)
    return df_results

In [6]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer


model_path = "./fine_tuned_model"


model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

training_args = TrainingArguments(
    output_dir="xlm-roberta-large-xquad",
    report_to="none"                
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [7]:

from evaluate import load
squad_metric = load("squad")

langs = ["de.de", "en.en", "es.es", "hi.hi"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

de.de


Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

en.en


Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

es.es


Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

hi.hi


Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [8]:

validation_results = compute_results(langs, split)


df_validation_results = results_df(validation_results, "fine_tuned_model")
print(df_validation_results)

de.de ['de.de', 'en.en', 'es.es', 'hi.hi']


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5278 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

en.en ['de.de', 'en.en', 'es.es', 'hi.hi']


...

  0%|          | 0/5253 [00:00<?, ?it/s]

hi.hi ['de.de', 'en.en', 'es.es', 'hi.hi']


Post-processing 4918 example predictions split into 6351 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

    lang  F1_fine_tuned_model  EM_fine_tuned_model
0  de.de                54.79                38.50
1  en.en                70.38                56.12
2  es.es                60.21                38.38
3  hi.hi                57.01                39.65
4    avg                60.60                43.16


In [9]:

df_validation_results.to_csv("results_fine_tuning_xquad_mlqa_xlm_r.csv")
df_validation_results

lang  F1_fine_tuned_model  EM_fine_tuned_model
0  de.de                54.79                38.50
1  en.en                70.38                56.12
2  es.es                60.21                38.38
3  hi.hi                57.01                39.65
4    avg                60.60                43.16